# Sign Langauge Dataset Collection

## 1. Download and Import all libraries

In [1]:
# !pip install -q mediapipe matplotlib scikit-learn numpy pandas tensorflow torch

In [2]:
import cv2
import numpy as np
import csv
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd
from random import sample
import zipfile
import json

In [3]:
# Initialize MediaPipe Hands and Pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [4]:
# Initialize MediaPipe Hands and Pose
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

In [5]:
def get_landmarks(frame_rgb):
    """
    Process the frame to extract hand and pose landmarks.
    :param frame_rgb: RGB frame from video
    :return: Hand landmarks, Pose landmarks
    """
    hand_results = hands.process(frame_rgb)
    return hand_results

In [6]:
def draw_landmarks(frame, hand_results):
    """
    Draw hand and pose landmarks on the frame.
    :param frame: Original frame from video
    :param hand_results: Hand landmarks from MediaPipe
    :return: Frame with drawn landmarks
    """
    # Draw hand landmarks
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
            )

    return frame

In [7]:
# Constants
NUM_LANDMARKS = 21
NUM_HANDS = 2
ZERO_LANDMARK = [[0, 0, 0]] * NUM_LANDMARKS  # Initialize placeholder for no landmarks

In [8]:
captured_landmarks = []  # List to store captured landmarks
captured_frames = []  # List for each frame's landmarks
frame_counter = 0  # Counter for frames

# Video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a selfie-view display
    frame = cv2.flip(frame, 1)

    # Resize the frame for processing
    # frame = cv2.resize(frame, (224, 224))

    # Convert frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get landmarks (assuming get_landmarks is defined)
    hand_results = get_landmarks(frame_rgb)
    cv2.putText(frame, 'Collection Number {}'.format(len(captured_landmarks)), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

    # Data capture logic
    if frame_counter == 0:
        # Reset captured_frames for this set of 50 frames
        captured_frames = []

    if hand_results and hand_results.multi_hand_landmarks:
        # Initialize frame landmarks with placeholders
        frame_landmarks = [ZERO_LANDMARK] * NUM_HANDS

        # Process detected hands
        for idx, hand_landmark in enumerate(hand_results.multi_hand_landmarks):
            if idx < NUM_HANDS:  # Limit to the number of hands expected
                # Convert each hand's landmarks to a list of [x, y, z] values (no array wrapper)
                frame_landmarks[idx] = [[lm.x, lm.y, lm.z] for lm in hand_landmark.landmark]

        # Append the processed landmarks for this frame (as lists, not arrays)
        captured_frames.append(frame_landmarks)
    else:
        # No hands detected, use placeholders
        captured_frames.append([ZERO_LANDMARK] * NUM_HANDS)

    frame_counter += 1

    # Save landmarks after collecting exactly 50 frames
    if frame_counter >= 50:
        captured_landmarks.append(captured_frames)
        captured_frames = []  # Reset the frames after saving
        frame_counter = 0  # Reset the counter after 30 frames are collected
        time.sleep(1)
    # Stop after capturing exactly 50 sets of 30 frames
    if len(captured_landmarks) >= 50:
        break

    # Draw landmarks (optional)
    frame = draw_landmarks(frame, hand_results)

    # Display the output video
    cv2.imshow('Hand Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [9]:
np.array(captured_landmarks).shape

(50, 50, 2, 21, 3)

In [10]:
def save_landmarks_with_class_csv(captured_landmarks, class_name, file_path='landmarks_data.csv'):
    """
    Save landmarks with the associated class name in CSV format (2 columns: class and landmarks).
    
    :param captured_landmarks: List of landmarks (frames with detected landmarks)
    :param class_name: Name of the class associated with the landmarks
    :param file_path: Path where the landmarks data will be saved
    """
    # Open the CSV file in append mode
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)

        # If the file is empty, write the header row
        if file.tell() == 0:
            header = ['class', 'landmarks']
            writer.writerow(header)

        # Loop through the captured landmarks and write each frame's data
        for frame_landmarks in captured_landmarks:
            # Convert the landmarks to a list and then to a string for saving (pure array format)
            landmarks_list = frame_landmarks  # Convert numpy array to list
            landmarks_str = str(landmarks_list)  # Convert the list to a string

            # Create a row with the class name and the string representation of the landmarks
            row = [class_name, landmarks_str]
            writer.writerow(row)

    print(f"Landmarks for class '{class_name}' have been saved to CSV.")

In [11]:
class_name = 'z'

# Call the function to save landmarks with class name in CSV
save_landmarks_with_class_csv(captured_landmarks, class_name)

Landmarks for class 'z' have been saved to CSV.


In [2]:
df = pd.read_csv('landmarks_data.csv')

NameError: name 'pd' is not defined

In [13]:
landmark = eval(df['landmarks'][0])

In [1]:
# Function to check the shape of landmarks
def check_landmark_shapes(landmark_data):
    # Convert the string back to a list using ast.literal_eval
    landmarks = ast.literal_eval(landmark_data)
    
    # Return the shape of the landmarks (a 2D list of landmarks for each hand in each frame)
    return len(landmarks), len(landmarks[0]) if landmarks else (0, 0)

# Iterate through the landmarks column to check the shapes
for index, row in df.iterrows():
    # Get the landmark data (assuming it is in the "landmarks" column)
    landmark_data = row['landmarks']
    
    # Check the shape of the current landmark data
    shape = check_landmark_shapes(landmark_data)
    print(f"Row {index} Landmark Shape: {shape}")

NameError: name 'df' is not defined